# HFK Caluculator for a knot represented by grid notation

## Modules

In [1]:
import numpy as np
import pandas as pd

from multiprocessing import Pool, cpu_count
from tqdm.notebook import tqdm

from GridPythonModule import *
from GPM_wrapper.Grid import *
import knot_floer_homology as hfk

## Input_grid_form Class

In [2]:
import ast
from ipywidgets import Text, HTML, Button, Output
from IPython.display import display, clear_output

class Input_grid_form():
    def __init__(self):
        self.grid = []
        self.xprm = Text(value='4,0,1,2,3',
                         placeholder='x marking 対応の置換を入力',
                         description='x の置換:',
                         disabled=False
                        )
        self.oprm = Text(value='1,2,3,4,0',
                         placeholder='o marking 対応の置換を入力',
                         description='o の置換:',
                         disabled=False
                        )
        self.start()
        
    def start(self):
        self.xprm.observe(get_list_str, names='value')
        self.oprm.observe(get_list_str, names='value')
        
        button = Button(description='Calculate HFK')
        output = Output(layour={'border': '1px solid black'})

        def on_click_callback(clicked_button: Button) -> None:
            self.grid = []
            with output:
                output.clear_output()
                self.grid = [ast.literal_eval(f"[{v}]") for v in [self.xprm.value, self.oprm.value]]
                print(f'\rGrid: {self.grid}', end="")
                try:
                    G = Grid(*self.grid)
                    G.draw()
                    PD = G.get_PD(); print(f"PD notation: {PD}\n")
                    hfk_dict = hfk.pd_to_hfk(PD)
                    tabulate_dict(hfk_dict)
                except Exception as e:
                    print(f" --> Error {e.args}", end="")

        # ボタンにイベントハンドラを登録する
        button.on_click(on_click_callback)
        display(self.xprm, self.oprm, button, output)

    def generate_random_grid(self):
        random
            
def get_list_str(arg):
    return f"[{arg}]"

def tabulate_dict(mydict):
    max_key_len = max(len(key) for key in mydict)
    max_value_len = max(len(str(val)) for val in mydict.values())
    print(f"{'Keys':>{max_key_len}} | {'Values':>{max_value_len}}")
    print('-' * (max_key_len + 3 + max_value_len))
    for key, value in mydict.items():
        print(f"{key:{max_key_len}} | {str(value):{max_value_len}}")

## Input a knot using grid notation

In [3]:
igform = Input_grid_form()

Text(value='4,0,1,2,3', description='x の置換:', placeholder='x marking 対応の置換を入力')

Text(value='1,2,3,4,0', description='o の置換:', placeholder='o marking 対応の置換を入力')

Button(description='Calculate HFK', style=ButtonStyle())

Output()

In [4]:
load_knot('8_16')

[[2, 1, 7, 8, 3, 5, 4, 9, 0, 10, 6], [9, 5, 10, 6, 7, 8, 2, 3, 4, 1, 0]]